In [ ]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


This code example contains a PyTorch implementation of a network classifying handwritten digits. More context for this code example can be found in video 3.6 "Programming Example: Digit Classification with PyTorch" in the video series "Learning Deep Learning: From Perceptron to Large Language Models" by Magnus Ekman (Video ISBN-13: 9780138177614).


The first code snippet contains initialization code for our PyTorch example. PyTorch has its own random seed that needs to be set if we want reproducible results. The code snippet also initializes a variable to indicate if there is a CUDA-capable GPU in the system.


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
torch.manual_seed(7)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 20
BATCH_SIZE = 1


We then load and prepare our MNIST dataset. Because MNIST is a common dataset, PyTorch contains a class that makes it easy to download and access the dataset. We can download and access it by a call to the constructor of the class MNIST. The parameters "download" and "root" describes that we want to download the dataset (if not already downloaded) and where to place it.

The code snippet contains a number of things worth discussing further. The parameter "train" in the constructor specifies if we want to access the training set or the test set. That is, we need to instantiate two MNIST objects. However, the code snippet below contains three calls to MNIST. The reason for that is that we want to first access the training dataset to compute the mean and standard deviation. That is the only usage of the first instantiation of MNIST in the code snippet.

In order to avoid having to loop through each element in the dataset we create a DataLoader object for the dataset. A DataLoader object is used to simplify accessing elements in the dataset. In particular, it is possible to specify a batch_size which determines how many elements the DataLoader will provide at a time. We set the batch_size to the full lenght of the dataset. That is, it will return an array containing all elements in the dataset. That makes it easy to compute the mean and standard deviation by simply calling mean() and stddev() on the full array.

Next we instantiate a transform object that specifies how we want to transform the data when when we load it. We state that we want to standardize the data. We also state that we want to convert the data into a tensor datatype, since PyTorch requires that for all data fed to a model.

We can now load and transform the training and test dataset into the right format needed for our network.


In [ ]:
# Load training dataset into a single batch to compute mean and stddev.
transform = transforms.Compose([transforms.ToTensor()])
trainset = MNIST(root='./pt_data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=True)
data = next(iter(trainloader))
mean = data[0].mean()
stddev = data[0].std()

# Helper function needed to standardize data when loading datasets.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, stddev)])

trainset = MNIST(root='./pt_data', train=True, download=True, transform=transform)
testset = MNIST(root='./pt_data', train=False, download=True, transform=transform)


We are now ready to create our network. There is no need to define variables for individual neurons because the framework provides functionality to instantiate entire layers of neurons at once.

The network is created by instantiating an nn.Sequential object, which implies that we are building a simple sequential network where layers are stacked on top of each other. We pass the layers as arguments to the Sequential class. The first layer is a Flatten layer, which does not do computations but only changes the organization of the input. In our case, the inputs are changed from a 28×28 array into an array of 784 elements. The second layer is a Linear layer, which means that it is fully connected. The first argument specifies the number of inputs and the second argument specifies the number of neurons. The activation function is declared as a separate layer, and we want to use tanh so we use a Tanh() layer. This is followed by another Linear layer and we end with a Sigmoid layer, which implements the logistic sigmoid function.

If we had done nothing else, the weights would be initialized using the default initialization scheme for each layer, but in our case we want to initialize them in the same way as we did in the stand-alone Python example (v3_3_2layer_learning_mnist). We do that by looping through all layers and for each Linear layer, we set them to uniform random values between -0.1 and 0.1. We set the bias weights to 0.


In [ ]:
# Create a Sequential (feed-forward) model.
# 784 inputs.
# Two fully-connected layers with 25 and 10 neurons.
# tanh as activation function for hidden layer.
# Logistic (sigmoid) as activation function for output layer.
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 25),
    nn.Tanh(),
    nn.Linear(25, 10),
    nn.Sigmoid()
)

# Initialize weights.
for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.uniform_(module.weight, a=-0.1, b=0.1)
        nn.init.constant_(module.bias, 0.0)


We are now ready to train the network, which is done in the next code snippet. In our opinion this is the hardest part when getting started with PyTorch since it requires a number of constructs applied in the right order. We will describe this training loop step by step. To a large extent, the mechanics of this training loop is the same regardless of what type of network we are building so in some of our later programming examples we build a helper function that can be reused and that hides much of the complexity.

We first create an optimizer object. In our case we use torch.optim.SGD, which means that we want to use stochastic gradient descent (SGD) when training the network. We specify the learning rate to be 0.01. We also create a loss function object, and in this case we use nn.MSELoss, which implies mean squared error. Note that although this example uses mean squared error (MSE) as loss function, in general that is not a good choice in combination with sigmoid output neurons. We change this in subsequent programming examples.

We then explicitly move the model from the CPU to the GPU, which is a requirement from PyTorch. If there is no CUDA-capable GPU available the variable "device" will be set to "cpu" (see initialization code above) so the .to() statements will not result in any movement.

We create a DataLoader object for both the training data and the test data. These objects contain functionality for accessing the dataset in the granularity of batch_size, and we can also specify if we want the DataLoader to radomize the order in which it returns examples. We specify that we want it to randomize the order of the training examples but not the test examples.

We then enter the training loop. Each iteration of the outer loop corresponds to a full epoch of training (going through the entire training set once). At the beginning of each iteration we set the model in training mode by calling model.train(). The reason for this is that certain types of layers need to behave differently during training than during test, and to do so, the layer needs to know if it is currently doing training or testing. Technically, the call to model.train() is unnecessary for in this example, because the layers used in this model behave the same both in training and test mode. We specify this anyway for good practice. Otherwise we risk spending time on chasing subtle bugs if we were to modify the model to use layers where this is needed.

We initialize the variables train_loss, train_correct, and train_batches. They will be used for tracking statistics of the training process.

Next we enter the inner training loop where each iteration corresponds to a mini-batch. The trainloader returns one batch of training input examples as well as the corresponding ground truth target values. We convert the target value to one-hot encoding and also convert to floating point values to match the type of value the model will produce.

We then explicitly move the data from the CPU to the GPU, just like we did for the model above.

Next we tell the optimizer to zero out all gradients. If this is not done, then previous training examples will affect the current mini-batch.

We run the forward pass by simply calling the model with the inputs as an argument, and it will return the computed outputs. We pass the outputs to the loss function, together with the ground truth targets.

The next step is to record metrics needed to later be able to report training loss and accuracy. The torch.max() function will return the index that has the highest value for each one-hot encoded output vector. That is, we end up with an array with the same number of elements as we have training examples in a mini-batch, and each element indicates what digit the network predicted. We compare if they are the same as the ground truth targets (the variable "targets" is not one-hot encoded) and then add up for how many this is the case (using the sum function). All of these computations were done on PyTorch tensors, but we want the final count to be a standard Python number. This conversion is done by calling the item(). Note that item() can only be called on a single number. If we want to convert a multi-element tensor into NumPy we need to call the numpy() method on the tensor instead. Apart from accumulating how many examples that were predicted correctly we also accumulate the total loss value.

Next we do the backward pass. The call to backward() will compute all partial derivatives. This is followed by a call to step(), which adjusts the weights.

Once we have looped through all mini-batches in the training set, we compute the overall accuracy and average loss by dividing the accumulated values by the number of mini-batches for the loss and the total number of training examples for the accuracy. The reason that the loss is not divided by the number of elements is that the loss reported by the loss function is already averaged over the number of elements in a mini-batch.

This concludes the training part of the outer loop. The second half of the outer loop largely does the same thing but for the training dataset. There are two key differences. The first is that we put the model in test mode by calling model.eval() instead of model.train(). Further, we do not do the backward pass and weight adjustment (we omit the calls to backward() and step()).

Finally, we print out accuracy and loss for both the training dataset and test dataset at the end of each epoch.


In [ ]:
# Use stochastic gradient descent (SGD) with
# learning rate of 0.01 and no other bells and whistles.
# MSE as loss function.
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

# Transfer model to GPU
model.to(device)

# Create DataLoader objects that will help create mini-batches.
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

# Train the model. In PyTorch we have to implement the training loop ourselves.
for i in range(EPOCHS):
    model.train() # Set model in training mode.
    train_loss = 0.0
    train_correct = 0
    train_batches = 0
    for inputs, targets in trainloader:
        # Move data to GPU.
        one_hot_targets = nn.functional.one_hot(targets, num_classes=10).float()
        inputs, targets, one_hot_targets = inputs.to(device), targets.to(device), one_hot_targets.to(device)

        # Zero the parameter gradients.
        optimizer.zero_grad()

        # Forward pass.
        outputs = model(inputs)
        loss = loss_function(outputs, one_hot_targets)

        # Accumulate metrics.
        _, indices = torch.max(outputs.data, 1)
        train_correct += (indices == targets).sum().item()
        train_batches +=  1
        train_loss += loss.item()

        # Backward pass and update.
        loss.backward()
        optimizer.step()

    train_loss = train_loss / train_batches
    train_acc = train_correct / (train_batches * BATCH_SIZE)

    # Evaluate the model on the test dataset. Identical to loop above but without
    # weight adjustment.
    model.eval() # Set model in inference mode.
    test_loss = 0.0
    test_correct = 0
    test_batches = 0
    for inputs, targets in testloader:
        one_hot_targets = nn.functional.one_hot(targets, num_classes=10).float()
        inputs, targets, one_hot_targets = inputs.to(device), targets.to(device), one_hot_targets.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, one_hot_targets)
        _, indices = torch.max(outputs, 1)
        test_correct += (indices == targets).sum().item()
        test_batches +=  1
        test_loss += loss.item()

    test_loss = test_loss / test_batches
    test_acc = test_correct / (test_batches * BATCH_SIZE)

    print(f'Epoch {i+1}/{EPOCHS} loss: {train_loss:.4f} - acc: {train_acc:0.4f} - val_loss: {test_loss:.4f} - val_acc: {test_acc:0.4f}')
